In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [25]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [26]:


# Load the parquet file into a PyArrow table
df = pd.read_csv('player_stats_all_time_DK_UD_added.csv')


In [27]:
df.shape

(134409, 56)

In [28]:
df = df[df['season_type'] == 'REG']

In [29]:
df = df[df['week'] < 18]

In [30]:
df.shape

(127594, 56)

In [31]:
df.head()

,Unnamed: 0,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,...,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,Fantasy Points Draftkings,Fantasy Points UD
0,0,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,1,...,0,0.0,0.052632,NaN,NaN,0,12.7,13.7,13.7,13.2
1,1,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,2,...,0,0.0,0.117647,NaN,NaN,0,5.1,8.1,8.1,6.6
2,2,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,MIA,1999,4,...,0,NaN,0.023810,NaN,NaN,0,0.2,0.2,0.2,0.2
3,3,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,7,...,0,0.0,0.050000,NaN,NaN,0,3.5,5.5,5.5,4.5
4,4,00-0000003,NaN,Abdul-Karim al-Jabbar,RB,RB,NaN,CLE,1999,8,...,0,NaN,NaN,NaN,NaN,0,3.9,3.9,3.9,3.9


# Overall Ranking

In [32]:
# Step 1: Group players by season and total up fantasy points
season_totals = df.groupby(['season', 'player_display_name'])['Fantasy Points Draftkings'].sum().reset_index()

# Step 2: Pick the player and get their 2024 season total
target_player = "Brock Bowers"  # Replace with your player's name
player_2024_points = season_totals[
    (season_totals['season'] == 2024) & (season_totals['player_display_name'] == target_player)
]['Fantasy Points Draftkings'].values

if len(player_2024_points) == 0:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = player_2024_points[0]

    # Step 3: For each season, rank where this total would fall
    rankings = []

    for season in sorted(season_totals['season'].unique()):
        season_df = season_totals[season_totals['season'] == season].copy()
        # Add a rank for where the target point total would fall in that season
        rank = (season_df['Fantasy Points Draftkings'] >= target_points).sum() + 1  # +1 if you want 1-based rank
        rankings.append((season, rank, len(season_df)))

    # Step 4: Convert to DataFrame for a nice table
    rankings_df = pd.DataFrame(rankings, columns=['season', 'rank_if_2024_points', 'total_players'])
    rankings_df['percentile'] = 100 * (1 - (rankings_df['rank_if_2024_points'] - 1) / rankings_df['total_players'])

    print(f"Where {target_player}'s 2024 total ({target_points:.1f} pts) would rank in each season:")
    print(rankings_df)


Where Brock Bowers's 2024 total (253.7 pts) would rank in each season:
    season  rank_if_2024_points  total_players  percentile
0     1999                   26            560   95.535714
1     2000                   39            553   93.128391
2     2001                   31            540   94.444444
3     2002                   40            543   92.817680
4     2003                   32            547   94.332724
5     2004                   34            565   94.159292
6     2005                   23            543   95.948435
7     2006                   29            535   94.766355
8     2007                   30            554   94.765343
9     2008                   31            554   94.584838
10    2009                   36            559   93.738819
11    2010                   35            574   94.076655
12    2011                   31            580   94.827586
13    2012                   42            599   93.155259
14    2013                   42            5

# Overall Ranking Excluding QB

In [33]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points Draftkings'].sum().reset_index()

target_player = "Brock Bowers"

# Get 2024 total points for target player
target_row = season_totals[
    (season_totals['season'] == 2024) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points Draftkings'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} DraftKings points in 2024.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        # Filter to non-QBs only
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] != 'QB')].copy()

        # Add the target player’s 2024 score to this season’s player pool
        all_points = season_df['Fantasy Points Draftkings'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points Draftkings', ascending=False).reset_index(drop=True)
        first_place_player = season_df_sorted.iloc[0]['player_display_name']
        first_place_points = season_df_sorted.iloc[0]['Fantasy Points Draftkings']

        if rank_position == 1:
            gap = target_points - season_df_sorted.iloc[1]['Fantasy Points Draftkings'] if total_players > 1 else 0
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((
            season,
            rank_position,
            # total_players,
           # percentile,
            gap_note,
            first_place_player
        ))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points', 
        #'total_non_QBs',
        #'percentile',
        'gap_vs_top', 'actual_season_leader'
    ])

    print("\nRankings if using 2024 DraftKings score in previous seasons (Non-QBs only):")
    print(rankings_df)


Brock Bowers scored 253.7 DraftKings points in 2024.

Rankings if using 2024 DraftKings score in previous seasons (Non-QBs only):
    season  rank_if_2024_points         gap_vs_top actual_season_leader
0     1999                   20  -146.6 behind 1st       Marshall Faulk
1     2000                   33  -217.8 behind 1st       Marshall Faulk
2     2001                   22  -187.0 behind 1st       Marshall Faulk
3     2002                   30  -218.0 behind 1st        Priest Holmes
4     2003                   26  -214.1 behind 1st  LaDainian Tomlinson
5     2004                   24  -125.9 behind 1st          Tiki Barber
6     2005                   20  -157.1 behind 1st      Shaun Alexander
7     2006                   22  -258.4 behind 1st  LaDainian Tomlinson
8     2007                   21  -186.6 behind 1st      Adrian Peterson
9     2008                   23  -149.4 behind 1st          Steve Smith
10    2009                   24  -241.7 behind 1st          Steve Smith
11    

In [34]:
print(player_2024_points)

[253.7]


# By Selected Position Group

In [35]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points Draftkings'].sum().reset_index()

target_player = "Brock Bowers"

target_row = season_totals[
    (season_totals['season'] == 2024) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points Draftkings'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} points in 2024 as a {target_position}.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] == target_position)].copy()

        # Calculate rank using rank() (1 = highest)
        all_points = season_df['Fantasy Points Draftkings'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points Draftkings', ascending=False).reset_index(drop=True)

        if rank_position == 1:
            second_place_points = season_df_sorted.iloc[1]['Fantasy Points Draftkings'] if total_players > 1 else 0
            gap = target_points - second_place_points
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            first_place_points = season_df_sorted.iloc[0]['Fantasy Points Draftkings']
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((season, rank_position, 
                        # total_players,
                        # percentile, 
                         gap_note))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points',
        #'total_players_at_position',
        #'percentile', 
        'gap_vs_top'
    ])

    print("\nRankings if using 2024 season score in previous seasons:")
    print(rankings_df)


Brock Bowers scored 253.7 points in 2024 as a TE.

Rankings if using 2024 season score in previous seasons:
    season  rank_if_2024_points          gap_vs_top
0     1999                    1  +37.5 ahead of 2nd
1     2000                    2    -23.3 behind 1st
2     2001                    1  +78.8 ahead of 2nd
3     2002                    1  +68.4 ahead of 2nd
4     2003                    1  +63.7 ahead of 2nd
5     2004                    3    -34.6 behind 1st
6     2005                    2    -17.4 behind 1st
7     2006                    1  +49.7 ahead of 2nd
8     2007                    3     -9.8 behind 1st
9     2008                    2    -17.1 behind 1st
10    2009                    3    -24.0 behind 1st
11    2010                    1  +43.7 ahead of 2nd
12    2011                    3    -92.2 behind 1st
13    2012                    1  +12.8 ahead of 2nd
14    2013                    2    -67.8 behind 1st
15    2014                    2    -21.7 behind 1st
16    20

In [36]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points Draftkings'].sum().reset_index()

target_player = "Brock Bowers"

target_row = season_totals[
    (season_totals['season'] == 2024) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points Draftkings'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} DraftKings points in 2024 as a {target_position}.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] == target_position)].copy()

        # Calculate rank using rank() (1 = highest)
        all_points = season_df['Fantasy Points Draftkings'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points Draftkings', ascending=False).reset_index(drop=True)
        first_place_player = season_df_sorted.iloc[0]['player_display_name']
        first_place_points = season_df_sorted.iloc[0]['Fantasy Points Draftkings']

        if rank_position == 1:
            gap = target_points - season_df_sorted.iloc[1]['Fantasy Points Draftkings'] if total_players > 1 else 0
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((
            season,
            rank_position,
            #total_players,
            #percentile,
            gap_note,
            first_place_player
        ))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points',
        #'total_players_at_position',
        #'percentile', 
        'gap_vs_top', 'actual_season_leader'
    ])

    print("\nRankings if using 2024 DraftKings score in previous seasons:")
    print(rankings_df)



Brock Bowers scored 253.7 DraftKings points in 2024 as a TE.

Rankings if using 2024 DraftKings score in previous seasons:
    season  rank_if_2024_points          gap_vs_top actual_season_leader
0     1999                    1  +37.5 ahead of 2nd        Tony Gonzalez
1     2000                    2    -23.3 behind 1st        Tony Gonzalez
2     2001                    1  +78.8 ahead of 2nd        Tony Gonzalez
3     2002                    1  +68.4 ahead of 2nd            Todd Heap
4     2003                    1  +63.7 ahead of 2nd        Tony Gonzalez
5     2004                    3    -34.6 behind 1st        Tony Gonzalez
6     2005                    2    -17.4 behind 1st        Antonio Gates
7     2006                    1  +49.7 ahead of 2nd        Antonio Gates
8     2007                    3     -9.8 behind 1st         Jason Witten
9     2008                    2    -17.1 behind 1st        Tony Gonzalez
10    2009                    3    -24.0 behind 1st         Dallas Clark
1

In [37]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points UD'].sum().reset_index()

target_player = "Brock Bowers"

target_row = season_totals[
    (season_totals['season'] == 2024) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points UD'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} UnderDog points in 2024 as a {target_position}.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] == target_position)].copy()

        # Calculate rank using rank() (1 = highest)
        all_points = season_df['Fantasy Points UD'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points UD', ascending=False).reset_index(drop=True)
        first_place_player = season_df_sorted.iloc[0]['player_display_name']
        first_place_points = season_df_sorted.iloc[0]['Fantasy Points UD']

        if rank_position == 1:
            gap = target_points - season_df_sorted.iloc[1]['Fantasy Points UD'] if total_players > 1 else 0
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((
            season,
            rank_position,
            #total_players,
            #percentile,
            gap_note,
            first_place_player
        ))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points',
        #'total_players_at_position',
        #'percentile', 
        'gap_vs_top', 'actual_season_leader'
    ])

    print("\nRankings if using 2024 UnderDog score in previous seasons:")
    print(rankings_df)



Brock Bowers scored 193.7 UnderDog points in 2024 as a TE.

Rankings if using 2024 UnderDog score in previous seasons:
    season  rank_if_2024_points          gap_vs_top actual_season_leader
0     1999                    1  +10.0 ahead of 2nd        Tony Gonzalez
1     2000                    2    -20.8 behind 1st        Tony Gonzalez
2     2001                    1  +48.3 ahead of 2nd        Tony Gonzalez
3     2002                    1  +42.9 ahead of 2nd            Todd Heap
4     2003                    1  +37.7 ahead of 2nd        Tony Gonzalez
5     2004                    3    -25.6 behind 1st        Tony Gonzalez
6     2005                    2    -20.9 behind 1st        Antonio Gates
7     2006                    1  +35.2 ahead of 2nd        Antonio Gates
8     2007                    3     -8.8 behind 1st         Jason Witten
9     2008                    2    -20.1 behind 1st        Tony Gonzalez
10    2009                    4    -28.0 behind 1st         Dallas Clark
11   

# Overall Ranking

In [38]:
# Step 1: Group players by season and total up fantasy points
season_totals = df.groupby(['season', 'player_display_name'])['Fantasy Points UD'].sum().reset_index()

# Step 2: Pick the player and get their 2024 season total
target_player = "Brock Bowers"  # Replace with your player's name
player_2024_points = season_totals[
    (season_totals['season'] == 2024) & (season_totals['player_display_name'] == target_player)
]['Fantasy Points UD'].values

if len(player_2024_points) == 0:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = player_2024_points[0]

    # Step 3: For each season, rank where this total would fall
    rankings = []

    for season in sorted(season_totals['season'].unique()):
        season_df = season_totals[season_totals['season'] == season].copy()
        # Add a rank for where the target point total would fall in that season
        rank = (season_df['Fantasy Points UD'] >= target_points).sum() + 1  # +1 if you want 1-based rank
        rankings.append((season, rank, len(season_df)))

    # Step 4: Convert to DataFrame for a nice table
    rankings_df = pd.DataFrame(rankings, columns=['season', 'rank_if_2024_points', 'total_players'])
    rankings_df['percentile'] = 100 * (1 - (rankings_df['rank_if_2024_points'] - 1) / rankings_df['total_players'])

    print(f"Where {target_player}'s 2024 total ({target_points:.1f} pts) would rank in each season:")
    print(rankings_df)


Where Brock Bowers's 2024 total (193.7 pts) would rank in each season:
    season  rank_if_2024_points  total_players  percentile
0     1999                   37            560   93.571429
1     2000                   49            553   91.320072
2     2001                   46            540   91.666667
3     2002                   54            543   90.239411
4     2003                   46            547   91.773309
5     2004                   54            565   90.619469
6     2005                   41            543   92.633517
7     2006                   42            535   92.336449
8     2007                   42            554   92.599278
9     2008                   48            554   91.516245
10    2009                   51            559   91.055456
11    2010                   49            574   91.637631
12    2011                   49            580   91.724138
13    2012                   56            599   90.818030
14    2013                   54            5

# Overall Ranking Excluding QB

In [39]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points UD'].sum().reset_index()

target_player = "Brock Bowers"

# Get 2024 total points for target player
target_row = season_totals[
    (season_totals['season'] == 2024) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points UD'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} UD points in 2024.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        # Filter to non-QBs only
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] != 'QB')].copy()

        # Add the target player’s 2024 score to this season’s player pool
        all_points = season_df['Fantasy Points UD'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points UD', ascending=False).reset_index(drop=True)
        first_place_player = season_df_sorted.iloc[0]['player_display_name']
        first_place_points = season_df_sorted.iloc[0]['Fantasy Points UD']

        if rank_position == 1:
            gap = target_points - season_df_sorted.iloc[1]['Fantasy Points UD'] if total_players > 1 else 0
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((
            season,
            rank_position,
           # total_players,
           # percentile,
            gap_note,
            first_place_player
        ))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points', 
        #'total_non_QBs',
        #'percentile', 
        'gap_vs_top', 'actual_season_leader'
    ])

    print("\nRankings if using 2024 UnderDog scoring in previous seasons (Non-QBs only):")
    print(rankings_df)


Brock Bowers scored 193.7 UD points in 2024.

Rankings if using 2024 UnderDog scoring in previous seasons (Non-QBs only):
    season  rank_if_2024_points         gap_vs_top actual_season_leader
0     1999                   25  -140.6 behind 1st       Marshall Faulk
1     2000                   35  -220.3 behind 1st       Marshall Faulk
2     2001                   29  -184.5 behind 1st       Marshall Faulk
3     2002                   37  -212.0 behind 1st        Priest Holmes
4     2003                   32  -214.3 behind 1st        Priest Holmes
5     2004                   37  -127.9 behind 1st          Tiki Barber
6     2005                   27  -175.6 behind 1st      Shaun Alexander
7     2006                   28  -259.4 behind 1st  LaDainian Tomlinson
8     2007                   28  -185.6 behind 1st      Adrian Peterson
9     2008                   30  -116.9 behind 1st          Steve Smith
10    2009                   32  -200.7 behind 1st          Steve Smith
11    2010    

In [40]:
print(player_2024_points)

[193.7]


# By Selected Position Group

In [41]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points UD'].sum().reset_index()

target_player = "Brock Bowers"

target_row = season_totals[
    (season_totals['season'] == 2024) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points UD'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} UnderDog points in 2024 as a {target_position}.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] == target_position)].copy()

        # Calculate rank using rank() (1 = highest)
        all_points = season_df['Fantasy Points UD'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points UD', ascending=False).reset_index(drop=True)
        first_place_player = season_df_sorted.iloc[0]['player_display_name']
        first_place_points = season_df_sorted.iloc[0]['Fantasy Points UD']

        if rank_position == 1:
            gap = target_points - season_df_sorted.iloc[1]['Fantasy Points UD'] if total_players > 1 else 0
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((
            season,
            rank_position,
            #total_players,
           # percentile,
            gap_note,
            first_place_player
        ))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points', 
        #'total_players_at_position',
       # 'percentile',
        'gap_vs_top', 'actual_season_leader'
    ])

    print("\nRankings if using 2024 UnderDog score in previous seasons:")
    print(rankings_df)



Brock Bowers scored 193.7 UnderDog points in 2024 as a TE.

Rankings if using 2024 UnderDog score in previous seasons:
    season  rank_if_2024_points          gap_vs_top actual_season_leader
0     1999                    1  +10.0 ahead of 2nd        Tony Gonzalez
1     2000                    2    -20.8 behind 1st        Tony Gonzalez
2     2001                    1  +48.3 ahead of 2nd        Tony Gonzalez
3     2002                    1  +42.9 ahead of 2nd            Todd Heap
4     2003                    1  +37.7 ahead of 2nd        Tony Gonzalez
5     2004                    3    -25.6 behind 1st        Tony Gonzalez
6     2005                    2    -20.9 behind 1st        Antonio Gates
7     2006                    1  +35.2 ahead of 2nd        Antonio Gates
8     2007                    3     -8.8 behind 1st         Jason Witten
9     2008                    2    -20.1 behind 1st        Tony Gonzalez
10    2009                    4    -28.0 behind 1st         Dallas Clark
11   

# Any Player Non- QB selection Template

In [42]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points UD'].sum().reset_index()

target_player = "JuJu Smith-Schuster"

# Get 2024 total points for target player
target_row = season_totals[
    (season_totals['season'] == 2018) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points UD'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} UD points in 2024.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        # Filter to non-QBs only
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] != 'QB')].copy()

        # Add the target player’s 2024 score to this season’s player pool
        all_points = season_df['Fantasy Points UD'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points UD', ascending=False).reset_index(drop=True)
        first_place_player = season_df_sorted.iloc[0]['player_display_name']
        first_place_points = season_df_sorted.iloc[0]['Fantasy Points UD']

        if rank_position == 1:
            gap = target_points - season_df_sorted.iloc[1]['Fantasy Points UD'] if total_players > 1 else 0
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((
            season,
            rank_position,
           # total_players,
           # percentile,
            gap_note,
            first_place_player
        ))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points', 
        #'total_non_QBs',
        #'percentile', 
        'gap_vs_top', 'actual_season_leader'
    ])

    print("\nRankings if using 2024 UnderDog scoring in previous seasons (Non-QBs only):")
    print(rankings_df)


JuJu Smith-Schuster scored 241.4 UD points in 2024.

Rankings if using 2024 UnderDog scoring in previous seasons (Non-QBs only):
    season  rank_if_2024_points         gap_vs_top actual_season_leader
0     1999                    9   -92.9 behind 1st       Marshall Faulk
1     2000                   16  -172.6 behind 1st       Marshall Faulk
2     2001                   11  -136.8 behind 1st       Marshall Faulk
3     2002                   16  -164.3 behind 1st        Priest Holmes
4     2003                   12  -166.6 behind 1st        Priest Holmes
5     2004                   13   -80.2 behind 1st          Tiki Barber
6     2005                   12  -127.9 behind 1st      Shaun Alexander
7     2006                   10  -211.7 behind 1st  LaDainian Tomlinson
8     2007                   12  -137.9 behind 1st      Adrian Peterson
9     2008                   14   -69.2 behind 1st          Steve Smith
10    2009                    9  -153.0 behind 1st          Steve Smith
11    2

In [43]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points Draftkings'].sum().reset_index()

target_player = "JuJu Smith-Schuster"

target_row = season_totals[
    (season_totals['season'] == 2018) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points Draftkings'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} DraftKings points in 2024 as a {target_position}.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] == target_position)].copy()

        # Calculate rank using rank() (1 = highest)
        all_points = season_df['Fantasy Points Draftkings'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points Draftkings', ascending=False).reset_index(drop=True)
        first_place_player = season_df_sorted.iloc[0]['player_display_name']
        first_place_points = season_df_sorted.iloc[0]['Fantasy Points Draftkings']

        if rank_position == 1:
            gap = target_points - season_df_sorted.iloc[1]['Fantasy Points Draftkings'] if total_players > 1 else 0
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((
            season,
            rank_position,
            #total_players,
            #percentile,
            gap_note,
            first_place_player
        ))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points',
        #'total_players_at_position',
        #'percentile', 
        'gap_vs_top', 'actual_season_leader'
    ])

    print("\nRankings if using 2024 DraftKings score in previous seasons:")
    print(rankings_df)


JuJu Smith-Schuster scored 321.9 DraftKings points in 2024 as a WR.

Rankings if using 2024 DraftKings score in previous seasons:
    season  rank_if_2024_points          gap_vs_top actual_season_leader
0     1999                    3    -53.0 behind 1st      Marvin Harrison
1     2000                    5    -26.9 behind 1st      Marvin Harrison
2     2001                    5    -47.8 behind 1st      Marvin Harrison
3     2002                    4    -92.3 behind 1st      Marvin Harrison
4     2003                    3    -79.1 behind 1st           Randy Moss
5     2004                    2    -29.1 behind 1st      Muhsin Muhammad
6     2005                    3    -44.9 behind 1st          Steve Smith
7     2006                    1  +29.9 ahead of 2nd      Marvin Harrison
8     2007                    4    -90.4 behind 1st           Randy Moss
9     2008                    4    -81.2 behind 1st          Steve Smith
10    2009                    3   -173.5 behind 1st          Steve 

In [44]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points Draftkings'].sum().reset_index()

target_player = "JuJu Smith-Schuster"

target_row = season_totals[
    (season_totals['season'] == 2018) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points Draftkings'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} DraftKings points in 2024 as a {target_position}.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] == "TE")].copy()

        # Calculate rank using rank() (1 = highest)
        all_points = season_df['Fantasy Points Draftkings'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points Draftkings', ascending=False).reset_index(drop=True)
        first_place_player = season_df_sorted.iloc[0]['player_display_name']
        first_place_points = season_df_sorted.iloc[0]['Fantasy Points Draftkings']

        if rank_position == 1:
            gap = target_points - season_df_sorted.iloc[1]['Fantasy Points Draftkings'] if total_players > 1 else 0
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((
            season,
            rank_position,
            #total_players,
            #percentile,
            gap_note,
            first_place_player
        ))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points',
        #'total_players_at_position',
        #'percentile', 
        'gap_vs_top', 'actual_season_leader'
    ])

    print("\nRankings if using 2024 DraftKings score in previous seasons:")
    print(rankings_df)


JuJu Smith-Schuster scored 321.9 DraftKings points in 2024 as a WR.

Rankings if using 2024 DraftKings score in previous seasons:
    season  rank_if_2024_points           gap_vs_top actual_season_leader
0     1999                    1  +105.7 ahead of 2nd        Tony Gonzalez
1     2000                    1  +137.9 ahead of 2nd        Tony Gonzalez
2     2001                    1  +147.0 ahead of 2nd        Tony Gonzalez
3     2002                    1  +136.6 ahead of 2nd            Todd Heap
4     2003                    1  +131.9 ahead of 2nd        Tony Gonzalez
5     2004                    1   +60.5 ahead of 2nd        Tony Gonzalez
6     2005                    1  +111.8 ahead of 2nd        Antonio Gates
7     2006                    1  +117.9 ahead of 2nd        Antonio Gates
8     2007                    1   +60.7 ahead of 2nd         Jason Witten
9     2008                    1  +114.0 ahead of 2nd        Tony Gonzalez
10    2009                    1   +60.4 ahead of 2nd    

In [45]:
import pandas as pd

# Get totals
season_totals = df.groupby(['season', 'player_display_name', 'position'])['Fantasy Points UD'].sum().reset_index()

target_player = "JuJu Smith-Schuster"

target_row = season_totals[
    (season_totals['season'] == 2018) &
    (season_totals['player_display_name'] == target_player)
]

if target_row.empty:
    print(f"{target_player} not found in 2024 data.")
else:
    target_points = target_row['Fantasy Points UD'].values[0]
    target_position = target_row['position'].values[0]

    print(f"{target_player} scored {target_points:.1f} UnderDog points in 2024 as a {target_position}.")

    rankings = []

    for season in sorted(season_totals['season'].unique()):
        season_df = season_totals[(season_totals['season'] == season) &
                                  (season_totals['position'] == "TE")].copy()

        # Calculate rank using rank() (1 = highest)
        all_points = season_df['Fantasy Points UD'].tolist() + [target_points]
        ranks = pd.Series(all_points).rank(ascending=False, method='min')
        rank_position = int(ranks.iloc[-1])

        total_players = len(season_df)
        percentile = 100 * (1 - (rank_position - 1) / total_players)

        season_df_sorted = season_df.sort_values(by='Fantasy Points UD', ascending=False).reset_index(drop=True)
        first_place_player = season_df_sorted.iloc[0]['player_display_name']
        first_place_points = season_df_sorted.iloc[0]['Fantasy Points UD']

        if rank_position == 1:
            gap = target_points - season_df_sorted.iloc[1]['Fantasy Points UD'] if total_players > 1 else 0
            gap_note = f"+{gap:.1f} ahead of 2nd"
        else:
            gap = first_place_points - target_points
            gap_note = f"-{gap:.1f} behind 1st"

        rankings.append((
            season,
            rank_position,
            #total_players,
            #percentile,
            gap_note,
            first_place_player
        ))

    rankings_df = pd.DataFrame(rankings, columns=[
        'season', 'rank_if_2024_points',
        #'total_players_at_position',
        #'percentile', 
        'gap_vs_top', 'actual_season_leader'
    ])

    print("\nRankings if using 2024 UnderDog score in previous seasons:")
    print(rankings_df)


JuJu Smith-Schuster scored 241.4 UnderDog points in 2024 as a WR.

Rankings if using 2024 UnderDog score in previous seasons:
    season  rank_if_2024_points          gap_vs_top actual_season_leader
0     1999                    1  +57.7 ahead of 2nd        Tony Gonzalez
1     2000                    1  +96.9 ahead of 2nd        Tony Gonzalez
2     2001                    1  +96.0 ahead of 2nd        Tony Gonzalez
3     2002                    1  +90.6 ahead of 2nd            Todd Heap
4     2003                    1  +85.4 ahead of 2nd        Tony Gonzalez
5     2004                    1  +26.5 ahead of 2nd        Tony Gonzalez
6     2005                    1  +75.8 ahead of 2nd        Antonio Gates
7     2006                    1  +82.9 ahead of 2nd        Antonio Gates
8     2007                    1  +44.7 ahead of 2nd         Jason Witten
9     2008                    1  +80.0 ahead of 2nd        Tony Gonzalez
10    2009                    1  +27.9 ahead of 2nd         Dallas Clar

# Now PPG For WR and TE ranked

In [46]:
df.columns

Index(['Unnamed: 0', 'player_id', 'player_name', 'player_display_name',
       'position', 'position_group', 'headshot_url', 'recent_team', 'season',
       'week', 'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', '

In [47]:
import pandas as pd
from tabulate import tabulate

# Filter for WRs in the 2024 season
wr_2024 = df[(df['position'] == 'WR') & (df['season'] == 2024)].copy()

# Calculate total fantasy points per player
total_points = wr_2024.groupby('player_name')['Fantasy Points Draftkings'].sum()

# Calculate games played per player (count non-null fantasy points)
games_played = wr_2024.groupby('player_name')['Fantasy Points Draftkings'].count()

# Compute Points Per Game (PPG)
ppg = total_points / games_played


# Create final DataFrame
top_wr_2024 = pd.DataFrame({
    'Player': total_points.index,  # Include player names
    'Total Points': total_points.values,
    'Games Played': games_played.values,
    'PPG': ppg.values
})

# Sort by PPG (highest first)
top_wr_2024 = top_wr_2024.sort_values(by='PPG', ascending=False)

# Display as a clean table
print("\n📊 Top WRs in 2024 (Points Per Game - DraftKings Scoring):\n")
print(tabulate(
    top_wr_2024.head(11),  # Show top 10 WRs
    headers='keys',
    tablefmt='fancy_grid',
    floatfmt=(".0f", "", ".0f", ".0f", ".2f")  # PPG with 2 decimal places
))



📊 Top WRs in 2024 (Points Per Game - DraftKings Scoring):

╒═════╤═════════════╤════════════════╤════════════════╤═══════╕
│     │ Player      │   Total Points │   Games Played │   PPG │
╞═════╪═════════════╪════════════════╪════════════════╪═══════╡
│  89 │ J.Chase     │            392 │             16 │ 24.52 │
├─────┼─────────────┼────────────────┼────────────────┼───────┤
│ 179 │ R.Rice      │             71 │              3 │ 23.63 │
├─────┼─────────────┼────────────────┼────────────────┼───────┤
│  31 │ C.Godwin    │            144 │              7 │ 20.54 │
├─────┼─────────────┼────────────────┼────────────────┼───────┤
│  98 │ J.Jefferson │            324 │             16 │ 20.25 │
├─────┼─────────────┼────────────────┼────────────────┼───────┤
│ 164 │ P.Nacua     │            222 │             11 │ 20.15 │
├─────┼─────────────┼────────────────┼────────────────┼───────┤
│ 194 │ T.Higgins   │            220 │             11 │ 19.98 │
├─────┼─────────────┼────────────────┼──────

In [48]:
top_wr_2024.to_csv("topWRs.csv")

In [49]:
import pandas as pd
from tabulate import tabulate

#df = df[df['week'] != 15]
#df = df[df['week'] != 14]
te_2024 = df[(df['position'] == 'TE') & (df['season'] == 2024)].copy()

total_points = te_2024.groupby('player_display_name')['Fantasy Points Draftkings'].sum()
games_played = te_2024.groupby('player_display_name')['Fantasy Points Draftkings'].count()

ppg = total_points / games_played

top_te_2024 = pd.DataFrame({
    'Player': total_points.index,  # Include player names
    'Total Points': total_points.values,
    'Games Played': games_played.values,
    'PPG': ppg.values
})

top_te_2024 = top_te_2024.sort_values(by='PPG', ascending=False)

print("\n📊 Top WRs in 2024 (Points Per Game - DraftKings Scoring):\n")
print(tabulate(
    top_te_2024.head(10),  # Show top 10 WRs
    headers='keys',
    tablefmt='fancy_grid',
    floatfmt=(".0f", "", ".0f", ".0f", ".2f")  # PPG with 2 decimal places
))



📊 Top WRs in 2024 (Points Per Game - DraftKings Scoring):

╒═════╤════════════════╤════════════════╤════════════════╤═══════╕
│     │ Player         │   Total Points │   Games Played │   PPG │
╞═════╪════════════════╪════════════════╪════════════════╪═══════╡
│  45 │ George Kittle  │            244 │             14 │ 17.42 │
├─────┼────────────────┼────────────────┼────────────────┼───────┤
│  11 │ Brock Bowers   │            254 │             16 │ 15.86 │
├─────┼────────────────┼────────────────┼────────────────┼───────┤
│ 108 │ Trey McBride   │            233 │             15 │ 15.55 │
├─────┼────────────────┼────────────────┼────────────────┼───────┤
│  30 │ David Njoku    │            148 │             11 │ 13.50 │
├─────┼────────────────┼────────────────┼────────────────┼───────┤
│ 103 │ Taysom Hill    │            107 │              8 │ 13.42 │
├─────┼────────────────┼────────────────┼────────────────┼───────┤
│  65 │ Jonnu Smith    │            210 │             16 │ 13.11 │
├─

In [50]:
top_te_2024.to_csv("topTEs.csv")